#Importing libraries and data

In [ ]:
!pip install osmnx
!pip install geopandas
!pip install pygeos

In [ ]:
!python -m pip uninstall matplotlib
!!pip install matplotlib==3.1.3

In [4]:
#mandatory for datascience and ploting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
#For geographical use
import geopandas as gpd
from ast import literal_eval
import osmnx as ox
from shapely.geometry import Point, LineString, shape

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [5]:
# reading road construction polygons
road_construction = pd.read_csv('hungary_road_construction.csv',sep=";",index_col=None,header=None)
road_construction.columns = ["name","year","month","day","points"]
road_construction.head()

,name,year,month,day,points
0,Polgári Tisza-hid,2002,10.0,1.0,"[[21.0552074392,47.8535623743],[21.0510017355,..."
1,Szekszard Szent Laszlo-hid,2003,7.0,4.0,"[[18.8875748594,46.3498790197],[18.887274452,4..."
2,Dunaujvaros Pentele-hid,2007,7.0,23.0,"[[18.9884702044,46.8892504416],[18.9010945635,..."
3,M3 Fuzesabony-Polgar,2002,NaN,NaN,"[[20.4282135249,47.7251911319],[20.4055542231,..."
4,M3 Polgar-Gorbehaza,2004,9.0,NaN,"[[21.2596564086,47.7818136227],[21.1813788207,..."


 we don't need the columns month and day so I will remove them!

In [6]:
road_construction.drop(columns=['month' , 'day'] , inplace=True)
#Years were string I changed them to int
road_construction.year=road_construction.year.map(int)
# converting list of list strings to actual Python objects
road_construction["points"] = road_construction["points"].map(literal_eval)

Load graph file from main roads data

In [8]:
#Load the data that Eszter prepared
Gs = ox.io.load_graphml(filepath='hungary_main_roads.graphml')
nodes, edges = ox.graph_to_gdfs(Gs)

Make the proper dataframe

In [ ]:
# Convert that to geodataframe and use shapley to convert roads to line string
gdf = gpd.GeoDataFrame(road_construction, crs={'init': 'EPSG:4326'}, geometry=road_construction['points'].apply(lambda x: LineString(x)))
#delete column which is not needed
gdf.drop(columns=['points'], inplace=True)
#Join left the constructions to bring the year column
intersections = gpd.sjoin(edges, gdf, how='left')
#Values that are NAN to -100000 to make it workable
intersections.year.fillna(-100000, inplace=True)


In [10]:
#make a directory for images
!mkdir data

In [25]:
#Make data from 2002 to 2012
for j in range(2002, 2013):
    edges_year = intersections[intersections.year<=j]
    graph=ox.graph_from_gdfs(nodes,edges_year)
    ox.io.save_graphml(graph,'data/hungary_road_network_%s.graphml'%j)

In [ ]:
#Make the zip from data folder
! zip -r /content/hungary_road_construction.zip /content/data